# CS421: Introduction to ML
## Team: G2T4
### Project: Predicting Credit Card Customer Churn from Spending Behaviours & Background
### Data Preprocessing
---

# 1. Setting up the notebook
---
### Importing libraries

In [40]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.model_selection import train_test_split

### Import dataset and tidy columns

In [41]:
data = pd.read_csv("../data/BankChurners.csv")

In [42]:
data = data.drop(columns=["CLIENTNUM", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"], axis=1)
data.head()

,Attrition_Flag,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,Total_Relationship_Count,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,Existing Customer,40,F,4,High School,Unknown,Less than $40K,Blue,34,3,4,1,3313.0,2517,796.0,1.405,1171,20,2.333,0.760
4,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000


In [4]:
# tidy up and standardise column naming convention
tidied_cols = []
for col_name in data.columns:
    col_name = col_name.lower()
    if '_ct' in col_name:
        col_name = col_name.replace('_ct', '_count')
    if '_chng' in col_name:
        col_name = col_name.replace('_chng', '_change')
    tidied_cols.append( col_name )

data.columns = tidied_cols.copy()
data.columns

Index(['attrition_flag', 'customer_age', 'gender', 'dependent_count',
       'education_level', 'marital_status', 'income_category', 'card_category',
       'months_on_book', 'total_relationship_count', 'months_inactive_12_mon',
       'contacts_count_12_mon', 'credit_limit', 'total_revolving_bal',
       'avg_open_to_buy', 'total_amt_change_q4_q1', 'total_trans_amt',
       'total_trans_count', 'total_count_change_q4_q1',
       'avg_utilization_ratio'],
      dtype='object')

In [5]:
# retrieve categorical and numerical columns
numerical = list(data.describe().columns)
categorical = [i for i in data.columns if i not in numerical and i != "attrition_flag"]

### Splitting the data into train and test splits

In [6]:
y = data[["attrition_flag"]]
x = data.drop("attrition_flag", axis=1)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2021)

# 2. Data Cleaning
---

### Impute unknown values

In [8]:
def impute_unknown(df_x):
    """
        Imputes unknown values with their mode
    """
    x = df_x.copy()

    marital_status_mode = x["marital_status"].mode()[0]
    x["marital_status"] = x["marital_status"].replace("Unknown", marital_status_mode)
    education_level_mode = x["education_level"].mode()[0]
    x["education_level"] = x["education_level"].replace("Unknown", education_level_mode)
    income_category_mode = x["income_category"].mode()[0]
    x["income_category"] = x["income_category"].replace("Unknown", income_category_mode)
    
    return x

In [9]:
x_train = impute_unknown(x_train)
x_test = impute_unknown(x_test)

# 3. Feature Engineering
---

In [10]:
def feature_engineering(df_x, df_y):
    """
        Encoding categorical columns
    """
    x = df_x.copy()
    y = df_y.copy()
    
    # Binary Encoding — features with binary categories  
    attrition_mapper = {"Existing Customer":0, "Attrited Customer":1}
    y["attrition_flag"] = y["attrition_flag"].replace(attrition_mapper)
    
    gender_mapper = {"F":0, "M":1}
    x["gender"] = x["gender"].replace(gender_mapper)
    
    # One Hot Encoding — features with nominal categories
    onehot_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
    values = onehot_enc.fit_transform(x[["marital_status"]])
    labels = np.array(["divorced", "married", "single"]).ravel()
    marital_status_df = pd.DataFrame(values, columns=labels)

    x.reset_index(drop=True, inplace=True)
    marital_status_df.reset_index(drop=True, inplace=True)
    x = pd.concat([x, marital_status_df], axis=1)
    x.drop("marital_status", axis=1, inplace=True)
    
    # Ordinal Encoding — features with ordinal categories
    edu_level_mapper = {"Doctorate":6, "Post-Graduate":5, "Graduate":4, "College":3, "High School":2, "Uneducated":1}
    x["education_level"] = x["education_level"].replace(edu_level_mapper)

    income_cat_mapper = {"$120K +":5, "$80K - $120K":4, "$60K - $80K":3, "$40K - $60K":2, "Less than $40K":1}
    x["income_category"] = x["income_category"].replace(income_cat_mapper)

    card_cat_mapper = {"Platinum":4, "Gold":3, "Silver":2, "Blue":1}
    x["card_category"] = x["card_category"].replace(card_cat_mapper)
    
    return x, y

In [11]:
x_train, y_train = feature_engineering(x_train, y_train)
x_test, y_test = feature_engineering(x_test, y_test)

# 4. Feature Selection
---

To reduce the dimensional space of the dataset, we are carrying out several feature selection techniques with the goal of identify features to keep and drop

1. Filter Methods
    - Chi-square
    - ANOVA
2. Embedded Methods
    - Random Forest
    - XGBoost

In [12]:
def retrieve_feature_importance(importance, names):
    """
        Helper function to display the feature importance
    """
    feature_importance = np.array(importance)
    feature_names = np.array(names)

    data = {'feature_name':feature_names,'feature_importance':feature_importance}
    feature_importance_df = pd.DataFrame(data)

    feature_importance_df.sort_values(by=['feature_importance'], ascending=False, inplace=True)
    return feature_importance_df.reset_index(drop=True)

## 4.1 Filter Methods
---
### 4.1.1 Chi-square

In [13]:
label_enc = LabelEncoder()
df_cat = data.copy()[categorical]

for cat in categorical:
    df_cat[cat] = label_enc.fit_transform(df_cat[cat])
    
y = data["attrition_flag"]
x = df_cat

In [14]:
test = SelectKBest(score_func=chi2, k='all')
test.fit(x, y)

results_df = retrieve_feature_importance(test.scores_, x.columns)
results_df.rename(columns={"feature_importance": "chi-square score"}, inplace=True)
display(results_df)

,feature_name,chi-square score
0,gender,7.443223
1,income_category,2.475170
2,marital_status,1.302755
3,card_category,0.986120
4,education_level,0.339231


### 4.1.2 ANOVA

In [15]:
x = data.copy()[numerical]
fscore = SelectKBest(k='all')
fscore.fit(x, y)

results_df = retrieve_feature_importance(fscore.scores_, x.columns)
results_df.rename(columns={"feature_importance": "f-score"}, inplace=True)
display(results_df)

,feature_name,f-score
0,total_trans_count,1620.121692
1,total_count_change_q4_q1,930.078416
2,total_revolving_bal,752.702408
3,contacts_count_12_mon,441.868050
4,avg_utilization_ratio,332.876795
5,total_trans_amt,296.227714
6,months_inactive_12_mon,240.910376
7,total_relationship_count,233.072886
8,total_amt_change_q4_q1,176.961638
9,credit_limit,5.773729


## 4.2 Embedded Methods
---

### 4.2.1 RandomForestClassifier

In [16]:
from sklearn.ensemble import RandomForestClassifier

rf_clf = RandomForestClassifier(n_estimators=1000, random_state=2021)
rf_clf.fit(x_train, y_train.values.ravel() )

RandomForestClassifier(n_estimators=1000, random_state=2021)

In [17]:
feat_impt_df = retrieve_feature_importance(rf_clf.feature_importances_, x_train.columns)
feat_impt_df

,feature_name,feature_importance
0,total_trans_amt,0.186250
1,total_trans_count,0.159180
2,total_revolving_bal,0.111792
3,total_count_change_q4_q1,0.108858
4,total_amt_change_q4_q1,0.066509
5,avg_utilization_ratio,0.064555
6,total_relationship_count,0.063362
7,credit_limit,0.034560
8,customer_age,0.034387
9,avg_open_to_buy,0.032576


### 4.2.2 XGBoost

In [18]:
from xgboost import XGBClassifier

xgb = XGBClassifier(eval_metric="logloss",
                    use_label_encoder=False,
                    random_state=2021)
                    
xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eval_metric='logloss',
              gamma=0, gpu_id=-1, importance_type='gain',
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, random_state=2021, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [19]:
feat_impt_df = retrieve_feature_importance(xgb.feature_importances_, x_train.columns)
feat_impt_df

,feature_name,feature_importance
0,total_trans_count,0.224598
1,total_revolving_bal,0.188462
2,total_relationship_count,0.127133
3,total_trans_amt,0.066519
4,total_count_change_q4_q1,0.057400
5,months_inactive_12_mon,0.045177
6,customer_age,0.042679
7,total_amt_change_q4_q1,0.032922
8,contacts_count_12_mon,0.027222
9,gender,0.026993


### Conclusion:
Results from the Feature Selection process are consolidated into this Google Sheet: https://docs.google.com/spreadsheets/d/1RnqAYQ1_cuOXaPCVTZo1uLN60a0qwHwB2qZ8l3BlKa8/edit?usp=sharing

In [20]:
x_train.drop(["total_trans_amt", "education_level", "single", "divorced", "married", "income_category", "card_category", "months_on_book", "avg_open_to_buy"], axis=1, inplace=True)
x_test.drop(["total_trans_amt", "education_level", "single", "divorced", "married", "income_category", "card_category", "months_on_book", "avg_open_to_buy"], axis=1, inplace=True)

# 5. Feature Transformation

In [21]:
def transformation(df_x):
    x = df_x.copy()
    
    # Feature Transformation — Log Transformation for skewed data
    skewed = ["credit_limit", "total_amt_change_q4_q1", "total_count_change_q4_q1"]

    for skewed_col in skewed:
        x[skewed_col] = np.log(x[skewed_col].mask(x[skewed_col] <= 0)).fillna(0)
    
    return x

x_train = transformation(x_train)
x_test = transformation(x_test)

# 6. Exporting preprocessed Train Test sets to csv files

In [22]:
x_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
train = pd.concat([x_train, y_train], axis=1)

x_test.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)
test = pd.concat([x_test, y_test], axis=1)

try:
    train.to_csv("../data/train.csv", index=False)
    test.to_csv("../data/test.csv", index=False)
    print('Succesfully exported')
except:
    print('Error')

Succesfully exported
